# RareSkills Zero Knowledge Week 1


## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [16]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

calculating 1590868 took 3.49 seconds


In [17]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

In [29]:
n = 9551
g = 5
s_p = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [30]:
def brute_force(s_p, g, n):
    for s in range(1, 9999):
        if pow(g, s, n) == s_p:
            return s


student_solution = brute_force(s_p, g, n)

In [31]:
assert pow(g, student_solution, n) == s_p
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [34]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [35]:
from sympy.ntheory import discrete_log


student_solution = discrete_log(n, encrypted_number, g)

In [36]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 420


## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers without knowing the numbers.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [52]:
n = 1000004119
g = 5
a = 5 # put your own value
b = 555 # put your own value
c = a + b

print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

False


Wait what?! That's supposed to be true?

<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

In [53]:
# Need to also % n the product of g**a * g**b

print((pow(g, a, n) * pow(g, b, n)) % n == pow(g, a + b, n))

True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [ ]:
g = 57
n = 101
g ** -5 % n

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [ ]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [ ]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

## Multiplication by a constant
Multiplication by a constant is really just (modern zk proof algorithms do not use this technique, this is just for illustration purposes. 4x is really x + x + x + x. In encrypted form, this would be

$$ g^{4a} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [ ]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [ ]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: Create a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>

In [80]:
# 2x + 8y = 7944
# 5x + 3y = 4764

# Solving the system of equations results in:
x = 420
y = 888

# g, n are agreed upon by both parties:
g = 57
n = 101

# Calculate the transformed x (A) and y (B)
A = pow(g, 2*x, n) # 1
B = pow(g, 8*y, n) # 87

# Calculate the solution C
C = pow(g, 7944, n) # 87

# A * B = C (mod n)
assert (A * B) % n == C

## The field of rational numbers ℚ is homomorphic to the field ℤp

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1023) in python. Show your work. You are not required to implement euclid's algorithm.**</font>

In [62]:
# (a + b) % p = (a%p + b%p)

p = 1033
a = (53 * pow(192, -1, 1033)) % p # 619
b = (61 * pow(511, -1, 1033)) % p # 928

print((a + b) % p) # 514

514
